# The Notebook goes through the testing did without imputing the data and just Dropping all the na values. 
### Import all the libraries and the data 

In [2]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import decomposition
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns

from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.decomposition import TruncatedSVD, FastICA, NMF, FactorAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV


from sklearn.manifold import TSNE

from sklearn.decomposition import KernelPCA, PCA

df = pd.read_csv("HumidityDataset.csv")

C:\Users\sherwin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (1,2,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.



In [3]:
df.head()

,station_id,longitude,latitude,time,AtmosphericPressure,WindDirection,WindSpeed,Gust,WaveHeight,WavePeriod,MeanWaveDirection,Hmax,AirTemperature,DewPoint,SeaTemperature,RelativeHumidity,QC_Flag
0,NaN,degrees_east,degrees_north,UTC,millibars,degrees true,knots,knots,meters,seconds,degrees_true,meters,degrees_C,degrees_C,degrees_C,percent,NaN
1,M1,-11.2,53.1266,2001-02-06T13:00:00Z,967.6,270.0,21.98,33.1,NaN,NaN,NaN,NaN,8.9,NaN,9.0,NaN,1.0
2,M1,-11.2,53.1266,2001-02-06T14:00:00Z,969.8,270.0,23.93,35.0,NaN,NaN,NaN,NaN,8.5,NaN,9.0,NaN,1.0
3,M1,-11.2,53.1266,2001-02-06T15:00:00Z,972.0,270.0,19.07,31.1,NaN,NaN,NaN,NaN,8.9,NaN,9.0,NaN,1.0
4,M1,-11.2,53.1266,2001-02-06T16:00:00Z,973.6,270.0,15.95,25.3,NaN,NaN,NaN,NaN,9.1,NaN,9.0,NaN,1.0


## Select only Feature Set 3 and drop the rest of the na values after selecting the features. 

In [4]:
df = df.iloc[331371:]
df.drop(["station_id","time", "WaveHeight", "WavePeriod", "MeanWaveDirection", "Hmax","QC_Flag"],inplace = True, axis = 1)
df = df[["AirTemperature","SeaTemperature" ,"DewPoint","RelativeHumidity"]]

In [5]:
df = df.apply(pd.to_numeric)
df = df.dropna()

In [6]:
df.corr()


,AirTemperature,SeaTemperature,DewPoint,RelativeHumidity
AirTemperature,1.000000,0.861137,0.900481,0.435637
SeaTemperature,0.861137,1.000000,0.688680,0.192427
DewPoint,0.900481,0.688680,1.000000,0.782621
RelativeHumidity,0.435637,0.192427,0.782621,1.000000


In [7]:
df.isna().sum()

AirTemperature      0
SeaTemperature      0
DewPoint            0
RelativeHumidity    0
dtype: int64

In [8]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df.iloc[:,:-1].values,df.iloc[:,-1].values, test_size=0.2)
print(X_train1.shape)

class CustomScaler():
    
        def __init__(self, X_Scale, y_Scale ) :
        
            self.X_Scale =   X_Scale 
            self.y_Scale =   y_Scale
            
            self.x_num =     [np.mean(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            self.x_maxs =    [np.max(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            self.x_mins =    [np.min(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            
            self.y_num =     np.mean(self.y_Scale)
            self.y_max_min = np.max(self.y_Scale) - np.min(self.y_Scale)

        def scaleX(self, x_value):
            x = x_value.copy()
            for i in range(x.shape[1]):
                
                x[:,i]= (x[:,i] - self.x_num[i])/(self.x_maxs[i]-self.x_mins[i])
            return x

        def inverseScaleX(self, x_value):
            x = x_value.copy()
            
            for i in range(x.shape[1]):
                x[:,i]= (x[:,i] * (self.x_maxs[i]-self.x_mins[i])) + self.x_num[i]
            return x



        

        def scaleY(self, y_value):
            y = y_value.copy()
            ys = (y - self.y_num)/(self.y_max_min)
            return ys

        def inverseScaleY(self, y_value):
            y = y_value.copy()
            ys = (y * self.y_max_min) + self.y_num
            return ys


        



custom_scaler = CustomScaler(X_train1,y_train1)
    
X_train = custom_scaler.scaleX(X_train1.copy())
y_train = custom_scaler.scaleY(y_train1.copy())
transform = None


print(X_train[-70000: -1])

print(X_test1[:10,:],y_test1[:10])



(82951, 3)
[[-0.12329128 -0.19399265 -0.18034898]
 [-0.07623246 -0.06006407 -0.1442044 ]
 [-0.03505599  0.06493593 -0.12412408]
 ...
 [ 0.17082636  0.1363645   0.22928957]
 [-0.0585854   0.02922164 -0.04781886]
 [ 0.07670872  0.10957878  0.04455062]]
[[11.6 12.2 11.4]
 [ 4.4 10.2  2.5]
 [12.3 13.5  8.2]
 [14.8 16.6 11.8]
 [ 4.8  9.5 -1.7]
 [12.4 12.9  8.1]
 [12.1 12.8  8.9]
 [ 9.8 10.8  6. ]
 [ 8.8  9.   7.6]
 [13.5 14.4  9.9]] [99. 87. 76. 82. 63. 75. 81. 77. 92. 79.]


## Model to Predict the Relative Humidity without Imputation 

In [10]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

adam  = tf.keras.optimizers.Adam(learning_rate=0.00001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)
adadelta = tf.keras.optimizers.Adadelta(lr=0.0001, rho=0.95, epsilon=1e-08, decay=0.0)
adagrad = tf.keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
rmsprop = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


def build_model(optimizer="adam", dropout_rate=0.5, activation = 'relu'):    
    model = keras.Sequential()
    model.add(Dense(100, activation = 'relu',input_dim = X_train.shape[1], name = "layer1"))
    model.add(Dense(200, activation = 'relu', name = "layer2"))
    model.add(Dense(100, activation = 'relu', name = "layer5"))
    model.add(Dense(1,  name = "layer7"))
    model.compile(loss="mean_squared_error", optimizer=adam)
    return model

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

regression_model = KerasRegressor(build_fn=build_model, epochs=50, batch_size=1024, verbose=1, validation_split = 0.2)
history=regression_model.fit(X_train, y_train)

# from sklearn.model_selection import cross_val_score
# regressor = KerasRegressor(build_fn = build_model, batch_size = 1024, epochs = 100)
# accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train,scoring='neg_mean_squared_error',cv = 5, n_jobs = 1)
# mean = accuracies.mean()
# variance = accuracies.std()
# print(mean)
# print(variance)


Epoch 1/50
65/65 [==============================] - 1s 13ms/step - loss: 0.0295 - val_loss: 0.0276
Epoch 2/50
65/65 [==============================] - 1s 10ms/step - loss: 0.0263 - val_loss: 0.0246
Epoch 3/50
65/65 [==============================] - 1s 11ms/step - loss: 0.0234 - val_loss: 0.0219
Epoch 4/50
65/65 [==============================] - 0s 7ms/step - loss: 0.0207 - val_loss: 0.0193
Epoch 5/50
65/65 [==============================] - 0s 6ms/step - loss: 0.0181 - val_loss: 0.0168
Epoch 6/50
65/65 [==============================] - 0s 7ms/step - loss: 0.0157 - val_loss: 0.0144
Epoch 7/50
65/65 [==============================] - 1s 10ms/step - loss: 0.0134 - val_loss: 0.0123
Epoch 8/50
65/65 [==============================] - 1s 11ms/step - loss: 0.0113 - val_loss: 0.0103
Epoch 9/50
65/65 [==============================] - 1s 11ms/step - loss: 0.0095 - val_loss: 0.0086
Epoch 10/50
65/65 [==============================] - 1s 9ms/step - loss: 0.0079 - val_loss: 0.0071
Epoch 11/50
6

## Evaluate the model 

In [12]:
from sklearn.metrics import mean_squared_error
transformer = None
if transformer:
    y_pred = regression_model.predict(transformer.transform(custom_scaler.scaleX(np.asarray(X_test1).astype('float32'))))
else :
    y_pred = regression_model.predict((custom_scaler.scaleX(np.asarray(X_test1).astype('float32'))))

# y_pred = max_min*y_pred + y_min
print(mean_squared_error(custom_scaler.inverseScaleY(y_pred), y_test1))


21/21 [==============================] - 0s 2ms/step
0.2642013296530774


In [13]:
np.column_stack((y_test1,custom_scaler.inverseScaleY(y_pred)))

array([[99.        , 98.10927582],
       [87.        , 86.67373657],
       [76.        , 75.67705536],
       ...,
       [94.        , 93.93611145],
       [83.        , 82.90410614],
       [69.        , 68.6388855 ]])